# Implementasi hasil VSM dengan algoritma Logistic Regression



memperbaiki load tfidf_vectorizer dan lr_model pada testing

* Nama  : Aurellia Zhullvita Amanullah
* NIM   : 220411100200
* Kelas : PPW A

# Import Library

In [ ]:
!pip install -q Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.2 MB/s eta 0:00:00


In [ ]:
# library awal untuk perhitungan dan pengolahan teks
import numpy as np
import re
import pandas as pd

# alat untuk crawling
from urllib.request import urlopen
from bs4 import BeautifulSoup

# monitoring
from tqdm import tqdm

# library untuk praproses teks
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# library untuk proses modeling
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# library untuk evaluasi model
from sklearn.metrics import classification_report, confusion_matrix

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# save model
import pickle


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Persiapan Data

Load Data Model

In [ ]:
main_df = pd.read_csv('https://raw.githubusercontent.com/aurelliazhullvita/kumpulan_tugas_ppw/refs/heads/master/data_beritakompas.csv', delimiter=',')
main_df

,No,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita
0,1,Simak Jadwal dan Lokasi SIM Keliling di Jakart...,"JAKARTA, KOMPAS.com - Surat Izin Mengemudi (S...",07/09/2024,OTOMOTIF
1,2,[POPULER OTOMOTIF] Diskon Motor Honda Septembe...,"JAKARTA, KOMPAS.com - Banyak pembaca yang ingi...",07/09/2024,OTOMOTIF
2,3,"Cek Saldo Minimal BRI, BNI, BCA, Mandiri, dan BSI","JAKARTA, KOMPAS.com - Penting bagi calon nasab...",06/09/2024,MONEY
3,4,"KAI Uji Coba Teknologi ""Face Recognition Board...",KOMPAS.com - PT Kereta Api Indonesia (KAI) Div...,06/09/2024,MONEY
4,5,OJK Blokir 10.890 Entitas Keuangan Ilegal Seja...,"JAKARTA, KOMPAS.com - Otoritas Jasa Keuangan (...",06/09/2024,MONEY
...,...,...,...,...,...
95,96,Waspada Masalah yang Timbul akibat Telat Ganti...,"JAKARTA, KOMPAS.com - Oli mesin pada mobil den...",06/09/2024,OTOMOTIF
96,97,"Sosok Faisal Basri di Mata Para Tokoh, Ekonom ...","JAKARTA, KOMPAS.com - Ekonom senior Faisal Bas...",06/09/2024,MONEY
97,98,"Pendaftaran CPNS Diperpanjang 4 Hari, Pelamar ...","JAKARTA, KOMPAS.com - Pemerintah telah memperp...",06/09/2024,MONEY
98,99,"Harga Emas Terbaru Pegadaian, Jumat 6 Septembe...","JAKARTA, KOMPAS.com - Pegadaian menyediakan be...",06/09/2024,MONEY


# Membuat Fungsi untuk Persiapan Crawling

In [ ]:
# fungsi untuk mengambil link yang akan dilakukan crawling
def extract_urls(url):
    html = urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    urls = soup.find_all("a", {"class": "paging__link"})
    urls = [url.get('href') for url in urls]

    return urls

# fungsi untuk mengambil isi dari berita
def get_content(url):
    html = urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    div = soup.find("div", {"class": "read__content"})
    paragraf = div.find_all("p")

    content = ''
    for p in paragraf:
        content += p.text

    return content


# fungsi utama crawling
def crawl(link = "https://indeks.kompas.com", max_money = 1, max_otomotif = 1, allow_category = ["OTOMOTIF", "MONEY"], is_train = True, title_old = []):
    # inisialisasi variabel penampung hasil berita
    news_data = []

    # inisialisasi persiapan untuk crawling berita
    last_url = extract_urls(link).pop()
    page = last_url.split('=').pop() # jumlah halaman secara otomatis
    # page = 1 # jumlah halaman secara manual

    # persiapan link yang akan dilakukan crawling
    urls = [link + '/?page=' + str(a) for a in range(1, int(page) + 1)]
    count_money = 0
    count_otomotif = 0

    # menelusuri semua link yang telah ditentukan
    for idx, url in enumerate(urls):
        if (len(news_data) == max_money + max_otomotif) :
          break

        html = urlopen(url).read()
        soup = BeautifulSoup(html, 'html.parser')

        # mengambil data yang diperlukan pada struktur html
        links       = soup.find_all("a", {"class": "article-link"})
        titles      = soup.find_all("h2", {"class": "articleTitle"})
        dates       = soup.find_all("div", {"class": "articlePost-date"})
        categories  = soup.find_all("div", {"class": "articlePost-subtitle"})

        news_per_page = len(links) # berita artikel yang ditampilkan

        # memasukkan data ke dalam list
        for elem in tqdm(range(news_per_page), desc=f"Crawling page {idx+1}"):
          news = {}
          category = categories[elem].text
          title = titles[elem].text

          if (category in allow_category):
            if (is_train):
              cond = (category == "MONEY" and count_money < max_money) or (category == "OTOMOTIF" and count_otomotif < max_otomotif)
            else:
              cond = (category == "MONEY" and count_money < max_money) or (category == "OTOMOTIF" and count_otomotif < max_otomotif) and title not in title_old


            if (cond):
              news['No'] = len(news_data) + 1
              news['Judul Berita']     = title
              news['Isi Berita']       = get_content(links[elem].get("href"))
              news['Tanggal Berita']   = dates[elem].text
              news['Kategori Berita']  = category
              news_data.append(news)

              if (category == "MONEY"):
                count_money += 1
              else:
                count_otomotif += 1

        print(f"=======> Money: {count_money} | Otomotif: {count_otomotif} | Total: {count_money + count_otomotif}")

    return news_data

Fungsi extract_urls digunakan untuk mengekstraksi link URL dengan pagination di halaman awal, sehingga menghasilkan beberapa URL yang dapat menuju ke halaman berikutnya atau sebelumnya.

Fungsi get_content digunakan untuk menghasilkan konten berita berdasarkan link yang dicari.

# Pengambilan Data Baru

In [ ]:
title_old = main_df["Judul Berita"].tolist()

test_news = crawl(max_money=5, max_otomotif=5, is_train=False, title_old=title_old)

Crawling page 1: 100%|██████████| 15/15 [00:01<00:00, 13.36it/s]


=======> Money: 1 | Otomotif: 0 | Total: 1


Crawling page 2: 100%|██████████| 15/15 [00:00<00:00, 15.24it/s]


=======> Money: 2 | Otomotif: 0 | Total: 2


Crawling page 3: 100%|██████████| 15/15 [00:00<00:00, 17265.25it/s]


...

=======> Money: 5 | Otomotif: 4 | Total: 9


Crawling page 15: 100%|██████████| 15/15 [00:00<00:00, 56679.78it/s]


=======> Money: 5 | Otomotif: 4 | Total: 9


Crawling page 16: 100%|██████████| 15/15 [00:00<00:00, 27.23it/s]

=======> Money: 5 | Otomotif: 5 | Total: 10


In [ ]:
main_df = pd.DataFrame(test_news)
main_df

,No,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita
0,1,"Prabowo Teken PP Hapus Tagih Utang UMKM, OJK: ...","JAKARTA, KOMPAS.com - Otoritas Jasa Keuangan (...",06/11/2024,MONEY
1,2,Mentan Amran: Penyaluran Pupuk Subsidi Harus S...,"JAKARTA, KOMPAS.com - Menteri Pertanian Andi A...",06/11/2024,MONEY
2,3,Tiket Kereta Api Periode Nataru 2024/2025 Suda...,KOMPAS.com - PT Kereta Api Indonesia (Persero)...,06/11/2024,MONEY
3,4,Kementan-Kemendes PDT Kolaborasi Percepatan Pr...,"JAKARTA, KOMPAS.com - Kementerian Pertanian da...",06/11/2024,MONEY
4,5,Apple Tawarkan Investasi Rp 157 Miliar untuk B...,"JAKARTA, KOMPAS.com - Kementerian Perindustri...",06/11/2024,MONEY
5,6,6 Tahapan Pertolongan Pertama Pasca Mobil Tere...,"KLATEN, KOMPAS.com - Kerusakan mobil akibat ba...",06/11/2024,OTOMOTIF
6,7,Gresini Racing Targetkan Podium Ganda di Seri ...,"JAKARTA, KOMPAS.com - Gresini Racing berhasil ...",06/11/2024,OTOMOTIF
7,8,Estimasi Biaya Perbaikan Mercedes-Benz jika Te...,"JAKARTA, KOMPAS.com – Menjelang akhir tahun, s...",06/11/2024,OTOMOTIF
8,9,Komunitas Raize dan Gazoo Racing Enthusiast Ge...,"JAKARTA, KOMPAS.com - Komunitas Toyota Raize I...",06/11/2024,OTOMOTIF
9,10,Jangan Langsung Nyalakan Mesin Setelah Mobil T...,"JAKARTA, KOMPAS.com - Beberapa daerah di Indon...",06/11/2024,OTOMOTIF


# Membuat Fungsi

In [ ]:
# Case Folding
def clean_lower(lwr):
    lwr = lwr.lower() # lowercase text
    return lwr

# Menghapus tanda baca, angka, dan simbol
def clean_punct(text):
    clean_spcl = re.compile('[/(){}\|@,;_]')
    clean_symbol = re.compile('[^0-9a-z]')
    clean_number = re.compile('[0-9]')
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    text = clean_number.sub('', text)
    return text

# Menghaps double atau lebih whitespace
def _normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")

# Menghapus stopwords
def clean_stopwords(text):
    stopword = set(stopwords.words('indonesian'))
    text = ' '.join(word for word in text.split() if word not in stopword) # hapus stopword dari kolom deskripsi
    return text

# Stemming with Sastrawi
def sastrawistemmer(text):
    factory = StemmerFactory()
    st = factory.create_stemmer()
    text = ' '.join(st.stem(word) for word in tqdm(text.split()) if word in text)
    return text

Fungsi clean_lower digunakan untuk mengonversi semua kata atau huruf menjadi huruf kecil.

Fungsi clean_punct berfungsi untuk menghapus karakter khusus, simbol, dan angka.

Fungsi _normalize_whitespace digunakan untuk menghilangkan spasi ganda atau lebih dari dua spasi.

Fungsi clean_stopwords digunakan untuk menghapus kata-kata yang tidak relevan (seperti kata sambung atau kata tambahan).

Fungsi sastrawistemmer digunakan dalam proses stemming untuk mengubah kata ke bentuk dasarnya.

# Clean Lower

In [ ]:
# Buat kolom tambahan untuk data description yang telah dilakukan proses case folding
main_df['lwr'] = main_df['Isi Berita'].apply(clean_lower)
casefolding=pd.DataFrame(main_df['lwr'])
casefolding

,lwr
0,"jakarta, kompas.com - otoritas jasa keuangan (..."
1,"jakarta, kompas.com - menteri pertanian andi a..."
2,kompas.com - pt kereta api indonesia (persero)...
3,"jakarta, kompas.com - kementerian pertanian da..."
4,"jakarta, kompas.com - kementerian perindustri..."
5,"klaten, kompas.com - kerusakan mobil akibat ba..."
6,"jakarta, kompas.com - gresini racing berhasil ..."
7,"jakarta, kompas.com – menjelang akhir tahun, s..."
8,"jakarta, kompas.com - komunitas toyota raize i..."
9,"jakarta, kompas.com - beberapa daerah di indon..."


# Clean Punct

In [ ]:
# Buat kolom tambahan untuk data description yang telah dilakukan proses penghapusan tanda baca
main_df['clean_punct'] = main_df['lwr'].apply(clean_punct)
main_df['clean_punct']


,clean_punct
0,jakarta kompas com otoritas jasa keuangan oj...
1,jakarta kompas com menteri pertanian andi am...
2,kompas com pt kereta api indonesia persero t...
3,jakarta kompas com kementerian pertanian dan...
4,jakarta kompas com kementerian perindustria...
5,klaten kompas com kerusakan mobil akibat ban...
6,jakarta kompas com gresini racing berhasil m...
7,jakarta kompas com menjelang akhir tahun sej...
8,jakarta kompas com komunitas toyota raize in...
9,jakarta kompas com beberapa daerah di indone...


# Normalize Whitespace

In [ ]:
main_df['clean_double_ws'] = main_df['clean_punct'].apply(_normalize_whitespace)
main_df['clean_double_ws']

,clean_double_ws
0,jakarta kompas com otoritas jasa keuangan ojk ...
1,jakarta kompas com menteri pertanian andi amra...
2,kompas com pt kereta api indonesia persero tel...
3,jakarta kompas com kementerian pertanian dan k...
4,jakarta kompas com kementerian perindustrian k...
5,klaten kompas com kerusakan mobil akibat banji...
6,jakarta kompas com gresini racing berhasil men...
7,jakarta kompas com menjelang akhir tahun sejum...
8,jakarta kompas com komunitas toyota raize indo...
9,jakarta kompas com beberapa daerah di indonesi...


# Clean Stopwords

In [ ]:
# Buat kolom tambahan untuk data description yang telah dilakukan proses penghapusan stopwords
main_df['clean_sw'] = main_df['clean_double_ws'].apply(clean_stopwords)
main_df['clean_sw']

,clean_sw
0,jakarta kompas com otoritas jasa keuangan ojk ...
1,jakarta kompas com menteri pertanian andi amra...
2,kompas com pt kereta api indonesia persero mem...
3,jakarta kompas com kementerian pertanian kemen...
4,jakarta kompas com kementerian perindustrian k...
5,klaten kompas com kerusakan mobil akibat banji...
6,jakarta kompas com gresini racing berhasil has...
7,jakarta kompas com menjelang wilayah indonesia...
8,jakarta kompas com komunitas toyota raize indo...
9,jakarta kompas com daerah indonesia memasuki m...


# Stemming dengan Sastrawi

In [ ]:
# Buat kolom tambahan untuk data description yang telah dilemmatization
main_df['desc_clean_stem'] = main_df['clean_sw'].apply(sastrawistemmer)
main_df['desc_clean_stem']


100%|██████████| 228/228 [00:04<00:00, 48.57it/s]


,desc_clean_stem
0,jakarta kompas com otoritas jasa uang ojk nila...
1,jakarta kompas com menteri tani andi amran sul...
2,kompas com pt kereta api indonesia persero buk...
3,jakarta kompas com menteri tani menteri desa b...
4,jakarta kompas com menteri industri kemenperin...
5,klaten kompas com rusa mobil akibat banjir par...
6,jakarta kompas com gresini racing hasil hasil ...
7,jakarta kompas com jelang wilayah indonesia gu...
8,jakarta kompas com komunitas toyota raize indo...
9,jakarta kompas com daerah indonesia pasuk musi...


# VSM

In [ ]:
# Load the saved model from file
filename = 'tfidf_vectorizer.sav'
tfidf_vectorizer = pickle.load(open(filename, 'rb'))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.3.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.3.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
corpus = main_df['desc_clean_stem']
tfidf = tfidf_vectorizer.transform(corpus)

In [ ]:
vocabulary = tfidf_vectorizer.get_feature_names_out().tolist()

tfidf_df = pd.DataFrame(tfidf.toarray(), columns=vocabulary)
tfidf_df.insert(0, 'Kategori Berita', main_df['Kategori Berita'])
tfidf_df

,Kategori Berita,aaion,aali,abadi,abai,abenkh,abnormal,absurd,ac,acapkali,...,za,zad,zag,zaman,zarco,zenix,zero,zig,zigzag,zona
0,MONEY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,MONEY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,MONEY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,MONEY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,MONEY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,OTOMOTIF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,OTOMOTIF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,OTOMOTIF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,OTOMOTIF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,OTOMOTIF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Preparing Data

# Encode Label

Encode Label adalah proses mengubah data kategorik (seperti teks atau label) menjadi bentuk numerik sehingga bisa digunakan dalam model machine learning. Ini bertujuan agar algoritma dapat memahami dan memproses data tersebut.

In [ ]:
# menggunakan label_encoder untuk merubah kata menjadi angka
label_encoder = preprocessing.LabelEncoder()
tfidf_df['Kategori Berita'] = label_encoder.fit_transform(tfidf_df['Kategori Berita'])

tfidf_df

,Kategori Berita,aaion,aali,abadi,abai,abenkh,abnormal,absurd,ac,acapkali,...,za,zad,zag,zaman,zarco,zenix,zero,zig,zigzag,zona
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Testing Data

In [ ]:
# Load the saved model from file
filename = 'lr_model.sav'
lr_model = pickle.load(open(filename, 'rb'))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.3.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
y_test = tfidf_df['Kategori Berita']
x_test = tfidf_df.drop(['Kategori Berita'], axis=1)
y_pred = lr_model.predict(x_test)

print(y_pred)

[0 0 0 0 0 1 1 1 1 1]


In [ ]:
# melihat nilai actual dan predicted
a = pd.DataFrame({'Actual value': y_test, 'Predicted value':y_pred})
a

,Actual value,Predicted value
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,1,1
6,1,1
7,1,1
8,1,1
9,1,1


# **Streamlit**

In [ ]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 5.1 MB/s eta 0:00:00


In [ ]:
import streamlit as st
import pandas as pd
import pickle
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Download NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

# Load pre-trained models (assuming model and vectorizer files are available)
try:
    with open("tfidf_vectorizer.sav", "rb") as f:
        tfidf_vectorizer = pickle.load(f)
    with open("lr_model.sav", "rb") as f:
        lr_model = pickle.load(f)
except FileNotFoundError:
    st.error("Model files not found. Please ensure 'tfidf_vectorizer.sav' and 'lr_model.sav' are in the same directory.")

# Set up Streamlit app
st.title("Text Classification with Logistic Regression")

st.write("""
### Upload your data for classification
""")

uploaded_file = st.file_uploader("Choose a CSV file", type="csv")
if uploaded_file is not None:
    data = pd.read_csv(uploaded_file)
    st.write("Data Preview:")
    st.write(data.head())

    # Check for the required text column
    if 'text_column' not in data.columns:
        st.error("The uploaded file must contain a column named 'text_column' for text data.")
    else:
        # Text preprocessing
        stop_words = set(stopwords.words('indonesian'))
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()

        def preprocess_text(text):
            # Lowercase
            text = text.lower()
            # Remove punctuation
            text = re.sub(r'\W', ' ', text)
            # Remove stopwords
            text = ' '.join([word for word in text.split() if word not in stop_words])
            # Stemming
            text = stemmer.stem(text)
            return text

        if st.button("Process and Classify Text"):
            # Apply preprocessing
            data['processed_text'] = data['text_column'].apply(preprocess_text)

            # Vectorize and classify
            text_features = tfidf_vectorizer.transform(data['processed_text'])
            predictions = lr_model.predict(text_features)
            data['Prediction'] = predictions

            st.write("Classification Results:")
            st.write(data[['text_column', 'Prediction']])

            # Display metrics if true labels are available
            if 'true_label' in data.columns:
                st.write("Classification Report:")
                st.text(classification_report(data['true_label'], predictions))
                st.write("Confusion Matrix:")
                st.write(confusion_matrix(data['true_label'], predictions))
            else:
                st.info("No 'true_label' column found. Only predictions are shown.")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2024-11-06 15:50:12.827 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-06 15:50:12.833 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-06 15:50:12.837 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-06 15:50:12.841 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-06 15:50:12.845 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-06 15:50:12.850 T